In [21]:
import pandas as pd
import os
from tqdm.notebook import tqdm
import json


# 경고 무시 모듈
import warnings


# 경고를 무시
warnings.filterwarnings(action='ignore')

# 경고 보이기  
#warnings.filterwarnings(action='default')

In [22]:
wifi = pd.read_csv('./원본/테마형특화단지ap_upper.csv', encoding='cp949')

In [26]:
# 위경도 csv 파일 읽기
wk = pd.read_csv('../전처리/wi-fi/테마형특화단지ap_위경도.csv', encoding='cp949', index_col=['MAC주소'])

In [127]:
cnct_df = None
for n in range(3,11):
    df = pd.read_csv(f'./원본/공공와이파이_데이터사용량/{n}월.csv', encoding='cp949')
    if n == 3:
        cnct_df = df
        continue
    cnct_df = pd.concat([cnct_df, df])

In [128]:
cnct_df = pd.DataFrame(cnct_df.groupby(['DEVICE Ascending'])['CLIENTS','BYTES USED (MB)'].sum())

In [129]:
cnct_df.to_csv('./원본/공공와이파이_데이터사용량/2022년.csv', encoding='cp949')

In [130]:
cnct_df

CLIENTS  BYTES USED (MB)
DEVICE Ascending                                           
Coffeeshop_SS_39lamp01                8625         73874.82
Coffeeshop_SS_39lamp02                7935          5639.10
Crossroad_gyeongwon_trafficpole01    27354        156460.02
Crossroad_gyeongwon_trafficpole02    13899         42381.33
Crossroad_naeoecenter_trafficpole    29422        123130.74
...                                    ...              ...
Tomb_kingsuro_exit                    2243        683374.85
Tomb_kingsuro_gate                    2011         32112.75
Tomb_kingsuro_infobox01                561        198220.40
Tomb_kingsuro_infobox02                878         73190.54
Touristinfo_gimhae_guideboard        15056        101714.53

[79 rows x 2 columns]

In [131]:
# 사용하는 wi-fi 추출

wifi_en_name = list(wifi['AP명'])
cnct_df = cnct_df.loc[wifi_en_name]
cnct_df

CLIENTS  BYTES USED (MB)
DEVICE Ascending                                           
Park_geobuk_CCTVpole                  9601         83306.56
Park_geobuk_basketball               16626        252943.92
Park_geobuk_trafficpole              18610        204907.80
Crossroad_gyeongwon_trafficpole01    27354        156460.02
Crossroad_gyeongwon_trafficpole02    13899         42381.33
...                                    ...              ...
Station_yeonjipark_bike              14633         79177.71
Street_haebancheon_trafficpole       20979         50750.51
Street_emartexit_trafficpole01       14936         36625.52
Street_emartexit_trafficpole02       19578         18888.43
Park_haebancheon_smartshadow          8585          9523.16

[74 rows x 2 columns]

In [132]:
cnct_df.reset_index(inplace=True)

In [133]:
# wi-fi 설명 & 위치와 데이터 사용량 결합

cnct_df['MAC'] = None
cnct_df['X'] = None
cnct_df['Y'] = None

for n in tqdm(range(len(cnct_df))):
    cnct_df['MAC'][n] = list(wifi[wifi['AP명']==cnct_df['DEVICE Ascending'][n]]['MAC주소'])[0]
    cnct_df['X'][n] = wk.loc[cnct_df['MAC'][n]]['경도']
    cnct_df['Y'][n] = wk.loc[cnct_df['MAC'][n]]['위도']

  0%|          | 0/74 [00:00<?, ?it/s]

In [134]:
cnct_df.sort_values('X', inplace=True)
cnct_df.reset_index(drop=True, inplace=True)

In [135]:
cnct_df.head(1)

DEVICE Ascending  CLIENTS  BYTES USED (MB)           MAC  \
0  Street_busanbank_trafficpole    20008       2650316.16  7C573CCF497E   

            X          Y  
0  128.865222  35.232194

In [136]:
cnct_df_g = cnct_df.groupby(['X', 'Y'])['CLIENTS', 'BYTES USED (MB)'].sum()
cnct_df_g.reset_index(inplace=True)
cnct_df_g.head()

X          Y  CLIENTS  BYTES USED (MB)
0  128.865222  35.232194    20008       2650316.16
1  128.865360  35.235990    16626        252943.92
2  128.865417  35.236306     9601         83306.56
3  128.865583  35.234611    23070        132204.07
4  128.866056  35.237194    18610        204907.80

In [137]:
cnct_df.head(1)

DEVICE Ascending  CLIENTS  BYTES USED (MB)           MAC  \
0  Street_busanbank_trafficpole    20008       2650316.16  7C573CCF497E   

            X          Y  
0  128.865222  35.232194

In [138]:
cnct_df_g['MAC'] = None
cnct_df_g['DEVICE Ascending'] = None

for n in range(len(cnct_df_g)):
    cnct_df_g['MAC'][n] = list(cnct_df[(cnct_df['X']==cnct_df_g['X'][n]) &
                                  (cnct_df['Y']==cnct_df_g['Y'][n])].head(1)['MAC'])[0]
    cnct_df_g['DEVICE Ascending'][n] = list(cnct_df[(cnct_df['X']==cnct_df_g['X'][n]) &
                                                    (cnct_df['Y']==cnct_df_g['Y'][n])].head(1)['DEVICE Ascending'])[0]

In [139]:
cnct_df_g = cnct_df_g[['DEVICE Ascending','MAC','X', 'Y','CLIENTS', 'BYTES USED (MB)']]

In [140]:
cnct_df_g['1user_used'] = cnct_df_g['BYTES USED (MB)'] / cnct_df_g['CLIENTS']

In [141]:
cnct_df_g

DEVICE Ascending           MAC           X          Y  \
0        Street_busanbank_trafficpole  7C573CCF497E  128.865222  35.232194   
1              Park_geobuk_basketball  7C573CCF44DA  128.865360  35.235990   
2                Park_geobuk_CCTVpole  7C573CCF47E6  128.865417  35.236306   
3    Firestation_naeoe119_trafficpole  7C573CCF454E  128.865583  35.234611   
4             Park_geobuk_trafficpole  7C573CCF42BE  128.866056  35.237194   
5      Street_macdonald_trafficpole01  7C573CCF4CCC  128.866278  35.232028   
6      Street_macdonald_trafficpole02  7C573CCF456E  128.866306  35.232167   
7   Crossroad_naeoecenter_trafficpole  7C573CCF465E  128.866722  35.234472   
8              Park_yeonji_westCCTV02  7C573CCF4AEE  128.867111  35.245583   
9   Crossroad_gyeongwon_trafficpole01  7C573CCF4518  128.867278  35.237083   
10      Street_deoksanapt_trafficpole  7C573CCF457A  128.867583  35.234250   
11          Street_naeoedong_NHbank02  7C573CCF4398  128.867583  35.238750   
12             Park_yeonji_38parklamp  7C573CCF45F4  128.867889  35.246833   
13          Street_naeoedong_NHbank01  7C573CCF4D74  128.868167  35.238944   
14              Park_yeonji_gymtoilet  7C573CCF4740  128.868250  35.244917   
15             Park_yeonji_clocktower  7C573CCF4526  128.868417  35.247250   
16                   Park_gongju_bike  7C573CCF4342  128.868472  35.234583   
17             Park_yeonji_windpole01  7C573CCF4A7E  128.868472  35.247056   
18              Park_yeonji_jogakpark  7C573CCF49C0  128.868750  35.245417   
19                   Park_yeonji_bike  7C573CCF455A  128.869556  35.244639   
20            Station_yeonjipark_bike  7C573CCF470A  128.869694  35.249417   
21         Park_yeonji_fountainoffice  7C573CCF43DE  128.870028  35.245778   
22             Park_yeonji_playground  7C573CCF44C0  128.870444  35.245194   
23     Street_emartexit_trafficpole01  7C573CCF4F78  128.871500  35.228722   
24       Park_haebancheon_smartshadow  7C573CCF4352  128.871778  35.241833   
25                Station_museum_bike  7C573CCF46C8  128.871806  35.240167   
26                 Museum_gimhae_bike  7C573CCF49AA  128.871944  35.243833   
27          Station_tombkingsuro_bike  7C573CCF45BA  128.872111  35.232889   
28             Museum_gimhae_22lamp01  7C573CCF4A6E  128.872389  35.241639   
29     Street_haebancheon_trafficpole  7C573CCF43EE  128.872583  35.230333   
30             Coffeeshop_SS_39lamp01  7C573CCF4ECC  128.872639  35.234444   
31      Museum_daeseongdong_gimainmul  7C573CCF484E  128.872639  35.238278   
32               School_gubong_12lamp  7C573CCF4D0C  128.872667  35.239806   
33           School_gubong_guideboard  7C573CCF4E8C  128.872889  35.239806   
34     Park_bonghwangdae_guideboard02  7C573CCF4C04  128.873111  35.232278   
35         Museum_daeseongdong_peak03  7C573CCF4CF0  128.873111  35.236472   
36      Street_passengerterminal_CCTV  7C573CCF48D4  128.873806  35.227167   
37            Terminal_shinsegae_bike  7C573CCFB76A  128.873926  35.227438   
38      Touristinfo_gimhae_guideboard  7C573CCF4320  128.874139  35.239000   
39   Park_bonghwangdae_drainagepump01  7C573CCF4AF2  128.874222  35.229806   
40       Tomb_daeseongdong_parklamp01  7C573CCF4AE0  128.874306  35.238139   
41       Park_bonghwangdae_parkinglot  7C573CCF47D0  128.874750  35.232611   
42          Station_bonghwang_3exit02  7C573CCF4AE6  128.875167  35.227111   
43       Tomb_daeseongdong_eastCCTV01  7C573CCF4B2E  128.875194  35.236278   
44       Tomb_daeseongdong_eastCCTV02  7C573CCF49F8  128.875278  35.236111   
45             Park_sureungwon_CCTV01  7C573CCF4D54  128.875500  35.236000   
46        Museum_gimhaefolk_rooftop02  7C573CCF4EA6  128.876278  35.234722   
47         Park_queensuro_flashlamp01  7C573CCF4F88  128.877361  35.242861   
48            Tomb_kingsuro_infobox01  D015A6C7F506  128.878028  35.234694   
49                 Tomb_kingsuro_gate  7C573CCF4AAE  128.878500  35.234056   
50             Park_tombkingsuro_lamp  7C57

In [142]:
cnct_df_g.to_csv('./전처리/와이파이_데이터사용량/데이터사용량.csv', encoding='cp949', index=False)